<a href="https://colab.research.google.com/github/anuraagshankar/Pasckathon_Bugslayers/blob/master/same_fill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, Normalizer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, precision_recall_curve, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.utils import class_weight
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import gc
import pdb
from xgboost import XGBRegressor
import os
from tqdm.notebook import tqdm
import pickle
import pdb

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!cp '/content/drive/My Drive/sepsis/same_fill/back_fill/dataset.csv'
!cp '/content/drive/My Drive/sepsis/same_fill/front_fill/dataset.csv' '/content/drive/My Drive/sepsis/same_fill/front_dataset.csv'

In [ ]:
!unzip /content/drive/My\ Drive/sepsis/training_setA.zip 
!unzip /content/drive/My\ Drive/sepsis/training_setB.zip

In [ ]:
DIR1 = '/content/training/'
DIR2 = '/content/training_setB/'
FDIR = '/content/drive/My Drive/sepsis/same_fill/front_fill/'
BDIR = '/content/drive/My Drive/sepsis/same_fill/back_fill/'

In [ ]:
files1 = os.listdir(DIR1)
files2 = os.listdir(DIR2)
cols = list(pd.read_csv(DIR1+files1[0],sep='|').columns)

In [ ]:
df1 = pd.read_csv(DIR1+files1[0], sep='|')
df1

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,1,0
1,75.0,96.0,NaN,111.0,75.00,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,2,0
2,66.0,94.0,NaN,103.0,71.67,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,3,0
3,72.0,91.0,NaN,108.0,74.67,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,4,0
4,77.0,98.0,36.89,113.0,81.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,5,0
5,74.0,85.0,36.89,105.0,74.33,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,6,0
6,68.0,98.0,36.78,101.0,69.67,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,7,0
7,65.0,96.0,NaN,118.0,79.33,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,8,0
8,64.0,95.0,NaN,129.0,82.33,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,9,0
9,63.0,95.0,NaN,123.0,78.33,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.73,1,NaN,NaN,-0.03,10,0


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HR                20 non-null     float64
 1   O2Sat             19 non-null     float64
 2   Temp              6 non-null      float64
 3   SBP               19 non-null     float64
 4   MAP               19 non-null     float64
 5   DBP               0 non-null      float64
 6   Resp              20 non-null     float64
 7   EtCO2             0 non-null      float64
 8   BaseExcess        0 non-null      float64
 9   HCO3              4 non-null      float64
 10  FiO2              0 non-null      float64
 11  pH                0 non-null      float64
 12  PaCO2             0 non-null      float64
 13  SaO2              0 non-null      float64
 14  AST               0 non-null      float64
 15  BUN               4 non-null      float64
 16  Alkalinephos      0 non-null      float64
 17 

In [ ]:
df2 = df1.fillna(method='ffill').fillna(method='bfill')
df2

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,75.0,100.0,36.89,103.5,52.5,NaN,18.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,1,0
1,75.0,100.0,36.89,103.5,52.5,NaN,18.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,2,0
2,75.0,100.0,36.89,101.0,55.0,NaN,19.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,3,0
3,73.0,100.0,36.89,107.0,51.0,NaN,18.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,4,0
4,72.0,100.0,36.89,110.0,56.0,NaN,18.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,5,0
5,75.5,100.0,36.89,116.0,60.0,NaN,17.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,6,0
6,73.0,99.5,36.78,113.5,59.0,NaN,21.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,7,0
7,73.0,100.0,36.78,97.0,51.0,NaN,17.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.7,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,8,0
8,74.0,99.0,36.78,99.0,50.0,NaN,20.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.4,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,9,0
9,72.0,99.0,36.78,99.0,51.0,NaN,15.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,7.6,113.0,1.1,NaN,98.0,NaN,2.1,3.4,3.8,NaN,NaN,27.4,9.7,23.0,12.1,NaN,166.0,85.74,0,NaN,NaN,-0.02,10,0


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HR                24 non-null     float64
 1   O2Sat             24 non-null     float64
 2   Temp              24 non-null     float64
 3   SBP               24 non-null     float64
 4   MAP               24 non-null     float64
 5   DBP               0 non-null      float64
 6   Resp              24 non-null     float64
 7   EtCO2             0 non-null      float64
 8   BaseExcess        0 non-null      float64
 9   HCO3              24 non-null     float64
 10  FiO2              0 non-null      float64
 11  pH                0 non-null      float64
 12  PaCO2             0 non-null      float64
 13  SaO2              0 non-null      float64
 14  AST               0 non-null      float64
 15  BUN               24 non-null     float64
 16  Alkalinephos      0 non-null      float64
 17 

In [ ]:
stats_df = pd.read_csv('/content/drive/My Drive/sepsis/stats.csv')
stats_df.head()

,column,mean,median
0,HR,84.581443,83.5
1,O2Sat,97.193955,98.0
2,Temp,36.977228,37.0
3,SBP,123.750465,121.0
4,MAP,82.400100,80.0


In [ ]:
cnt = 1
for f in tqdm(files1):
  df = pd.read_csv(DIR1+f, sep='|')
  f_df = df.fillna(method='ffill').fillna(method='bfill')
  b_df = df.fillna(method='bfill').fillna(method='ffill')
  na_cols = f_df.columns[f_df.isna().any()].tolist()
  for c in na_cols:
    val = float(stats_df[stats_df['column']==c]['mean'])
    f_df[c].fillna(val, inplace=True)
    b_df[c].fillna(val, inplace=True)
  f_df.to_csv(f'/content/drive/My Drive/sepsis/same_fill/front_fill/{cnt}.csv',index=False)
  b_df.to_csv(f'/content/drive/My Drive/sepsis/same_fill/back_fill/{cnt}.csv',index=False)
  cnt += 1
for f in tqdm(files2):
  df = pd.read_csv(DIR2+f, sep='|')
  f_df = df.fillna(method='ffill').fillna(method='bfill')
  b_df = df.fillna(method='bfill').fillna(method='ffill')
  na_cols = f_df.columns[f_df.isna().any()].tolist()
  for c in na_cols:
    val = float(stats_df[stats_df['column']==c]['mean'])
    f_df[c].fillna(val, inplace=True)
    b_df[c].fillna(val, inplace=True)
  f_df.to_csv(f'/content/drive/My Drive/sepsis/same_fill/front_fill/{cnt}.csv',index=False)
  b_df.to_csv(f'/content/drive/My Drive/sepsis/same_fill/back_fill/{cnt}.csv',index=False)
  cnt += 1

In [ ]:
f_df = pd.concat([pd.read_csv(FDIR+f) for f in tqdm(os.listdir(FDIR))])
f_df.to_csv(FDIR+'dataset.csv',index=False)
b_df = pd.concat([pd.read_csv(BDIR+f) for f in tqdm(os.listdir(BDIR))])
b_df.to_csv(BDIR+'dataset.csv',index=False)

# **BASELINE MODEL**

In [ ]:
f_df = pd.read_csv('/content/drive/My Drive/sepsis/same_fill/front_dataset.csv')

In [ ]:
f_df = f_df.drop(columns=['Gender', 'Unit1', 'Unit2', 'BaseExcess', 'HospAdmTime', 'EtCO2'])

In [ ]:
cols = list(f_df.columns)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(f_df)
base_df = scaler.transform(f_df)
base_df = pd.DataFrame(base_df, columns = cols)
print(base_df.shape)

(1552210, 35)


In [ ]:
X = base_df.drop('SepsisLabel', axis=1)
y = base_df['SepsisLabel']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=50, class_weight='balanced')
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
print(roc_auc_score(y_test, y_pred))

0.8762137745741346


In [ ]:
b_df = pd.read_csv('/content/drive/My Drive/sepsis/same_fill/back_dataset.csv')

In [ ]:
b_df = b_df.drop(columns=['Gender', 'Unit1', 'Unit2', 'BaseExcess', 'HospAdmTime', 'EtCO2'])

In [ ]:
cols = b_df.columns

In [ ]:
scaler = MinMaxScaler()
scaler.fit(b_df)
base_df = scaler.transform(b_df)
base_df = pd.DataFrame(base_df, columns = cols)
print(base_df.shape)

(1552210, 35)


In [ ]:
cols

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'HCO3', 'FiO2',
       'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium',
       'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate',
       'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI',
       'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'ICULOS',
       'SepsisLabel'],
      dtype='object')

In [ ]:
X = base_df.drop('SepsisLabel', axis=1)
y = base_df['SepsisLabel']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=50, class_weight='balanced')
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
print(roc_auc_score(y_test, y_pred))

0.9042789253301867


In [ ]:
corrmatrix=f_df.corr()
print(corrmatrix)

                        HR     O2Sat  ...    ICULOS  SepsisLabel
HR                1.000000 -0.069410  ...  0.046840     0.049836
O2Sat            -0.069410  1.000000  ... -0.041587    -0.009278
Temp              0.228196  0.002226  ...  0.089230     0.050597
SBP              -0.036822  0.040054  ...  0.037895    -0.012318
MAP               0.062914  0.034521  ...  0.020791    -0.018989
DBP               0.108629  0.009724  ...  0.006946    -0.015106
Resp              0.218303 -0.134108  ...  0.092026     0.041619
EtCO2             0.002383  0.001281  ...  0.068573    -0.006416
BaseExcess       -0.013219  0.015112  ...  0.098510     0.014796
HCO3             -0.037451 -0.035728  ...  0.067134    -0.000053
FiO2              0.001258  0.000017  ... -0.006488    -0.000051
pH               -0.016687  0.062717  ...  0.154366     0.016553
PaCO2            -0.001166 -0.079144  ... -0.012744    -0.003425
SaO2              0.008410  0.046484  ...  0.024440     0.006367
AST               0.01283

In [ ]:
corrmatrix1=b_df.corr()
print(corrmatrix1)

                        HR     O2Sat  ...    ICULOS  SepsisLabel
HR                1.000000 -0.069211  ...  0.047672     0.050880
O2Sat            -0.069211  1.000000  ... -0.041633    -0.008304
Temp              0.231440  0.009163  ...  0.084924     0.056379
SBP              -0.037221  0.039384  ...  0.038265    -0.012442
MAP               0.062421  0.034851  ...  0.020575    -0.019539
DBP               0.107852  0.009755  ...  0.004470    -0.015386
Resp              0.218133 -0.135058  ...  0.091610     0.040945
EtCO2             0.001175 -0.008211  ...  0.057168    -0.006198
BaseExcess       -0.006509  0.019953  ...  0.100151     0.011264
HCO3             -0.022188 -0.031030  ...  0.067011    -0.007272
FiO2              0.001010 -0.002073  ... -0.004468     0.000289
pH               -0.010265  0.067885  ...  0.143261     0.013219
PaCO2             0.002958 -0.077310  ...  0.002746    -0.003680
SaO2              0.005449  0.051744  ...  0.023240     0.005616
AST               0.01340

In [ ]:
corrmatrix.to_csv(FDIR+'same_front_corr.csv')

In [ ]:
corrmatrix1.to_csv(BDIR+'same_back_corr.csv')